In [1]:
import sys
import json
import csv
import yaml

import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import math

import matplotlib as mpl
import matplotlib.pyplot as plt

import itertools

import time
import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

sys.path.append('/Users/Nfaith21/Documents/ECS 116 - Misc/DISC_5_FILES/benchmarking/')
import util_actual_main as util

In [2]:
dotenv_path = 'variables.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:

load_dotenv()

schema = os.getenv('DISC_4_SCHEMA')
port = os.getenv('DISC_4_PORT')
host = os.getenv('DISC_4_HOST')
database = os.getenv('DISC_4_DB')
username = os.getenv('username')
password = os.getenv('password')

In [4]:
db_eng = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(f"{schema}")},
                       isolation_level = 'SERIALIZABLE')

print("Successfully created db engine.")

Successfully created db engine.


# Step 5:

# Q3

In [5]:
def rename_keys(d):
    return {k.split('_')[-1] : v for k, v in d.items()}

def extract_value(data, val):
    values = {}
    for year, details in data.items():
        values[year] = {}
        for key, metrics in details.items():
            if val in metrics:
                values[year][key] = metrics[val]
    return values

In [6]:
data_update_datetimes = util.fetch_perf_data('update_datetimes_query.json')

In [7]:
axis_order = ['New Springville', 'Fort Hamilton', 'Long Island City', 'Bedford-Stuyvesant', 'Staten Island', 'Bronx', 'Queens', 'Manhattan']
renamed_data_update_datetimes = rename_keys(data_update_datetimes)
update_datetimes_avg = extract_value(renamed_data_update_datetimes, 'avg')
update_datetimes_avg_df = pd.DataFrame(update_datetimes_avg).T
update_datetimes_avg_df = update_datetimes_avg_df.reindex(axis_order)
update_datetimes_avg_df.rename(columns = {"__": "no_index",
    "__datetime_in_reviews__": "datetime_index",
    "__neigh_in_listings__": "neigh_index",
    "__datetime_in_reviews__neigh_in_listings__": "datetime_and_neigh_index"}
    , inplace = True)

In [8]:
update_datetimes_avg_df['Difference'] = update_datetimes_avg_df['datetime_and_neigh_index'] - update_datetimes_avg_df['no_index']

In [9]:
df_diff = pd.DataFrame(update_datetimes_avg_df['Difference'].sort_values())
df_diff

,Difference
Staten Island,-1.8522
New Springville,-0.7633
Fort Hamilton,-0.2123
Long Island City,-0.1407
Queens,1.8179
Bronx,2.9361
Bedford-Stuyvesant,12.5671
Manhattan,46.2367


In [10]:
query = """select l.neighbourhood as loc, count(*)
from reviews r, listings l
where l.id = r.listing_id
 and l.neighbourhood in ('New Springville', 'Fort Hamilton', 'Long Island City', 'Bedford-Stuyvesant')
group by loc
union
select l.neighbourhood_group as loc, count(*)
from reviews r, listings l
where l.id = r.listing_id
 and l.neighbourhood_group in ('Staten Island', 'Bronx', 'Queens', 'Manhattan')
group by loc
order by count;"""

with db_eng.connect() as conn:
    df_records = pd.read_sql_query(sql_text(query), conn)

In [11]:
df_records.index = df_records['loc']
df_records.drop(columns = 'loc', inplace = True)
df_records

,count
loc,
New Springville,104
Fort Hamilton,1000
Long Island City,10859
Staten Island,13726
Bronx,35296
Bedford-Stuyvesant,99705
Queens,173392
Manhattan,341287


In [12]:
pd.concat([df_records, df_diff], axis=1)

,count,Difference
New Springville,104,-0.7633
Fort Hamilton,1000,-0.2123
Long Island City,10859,-0.1407
Staten Island,13726,-1.8522
Bronx,35296,2.9361
Bedford-Stuyvesant,99705,12.5671
Queens,173392,1.8179
Manhattan,341287,46.2367


# Q4

In [13]:
data_listings = util.fetch_perf_data('listings_join_reviews.json')
data_listings = data_listings['q_listings_join_reviews_2019']

data_listings_no_index = data_listings['__']
data_listings_index = data_listings['__datetime_in_reviews__']

data_listings_no_index_avg = data_listings_no_index['avg']
data_listings_index_avg = data_listings_index['avg']

In [14]:
data_listings_no_index_avg, data_listings_index_avg

(2.9472, 0.2215)

In [15]:
# time saved by using index for 1 query
query_time_saved = data_listings_no_index_avg - data_listings_index_avg
print(f"Query: time saved by using index: {query_time_saved:.3f} seconds")

Query: time saved by using index: 2.726 seconds


In [16]:
# time saved by using index for 1000 queries
query_time_saved_1000 = query_time_saved * 1000
# convert to minutes and second
minutes = query_time_saved_1000 // 60
seconds = query_time_saved_1000 % 60
print(f"Query: time saved by using index for 1000 queries: {minutes:.0f} minutes {seconds:.2f} seconds")

Query: time saved by using index for 1000 queries: 45 minutes 25.70 seconds


In [17]:
data_update_manhattan = data_update_datetimes['update_datetimes_query_Manhattan']

data_update_no_index = data_update_manhattan['__']
data_update_index = data_update_manhattan['__datetime_in_reviews__']

data_update_no_index_avg = data_update_no_index['avg']
data_update_index_avg = data_update_index['avg']

In [18]:
data_update_no_index_avg, data_update_index_avg

(68.0796, 116.6993)

In [19]:
# time lost by using index for 1 update
update_time_lost = data_update_index_avg - data_update_no_index_avg
print(f"Update: time lost by using index: {update_time_lost:.4f} seconds")

Update: time lost by using index: 48.6197 seconds


In [20]:
# time lost by using index for 1000 updates
update_time_lost_1000 = update_time_lost * 1000
# convert seconds to hours: mintues: seconds
hours = update_time_lost_1000 // 3600
minutes = (update_time_lost_1000 % 3600) // 60
seconds = update_time_lost_1000 % 60
print(f"Update: time lost by using index for 1000 updates: {hours:.0f} hours, {minutes:.0f} minutes, {seconds:.1f} seconds")

Update: time lost by using index for 1000 updates: 13 hours, 30 minutes, 19.7 seconds


In [21]:
x = update_time_lost / (query_time_saved + update_time_lost)
print(f'x = {x:.4f}')

x = 0.9469
